In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
Grocery_json = pd.read_json('Resources/Grocery_and_Gourmet_Food.json',lines=True, nrows=10)


In [3]:
Grocery_json

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400
5,1,True,"02 21, 2014",ATQL0XOLZNHZ4,1888861614,Brianna Small,"The S is beautiful, however upon checkout it i...",shipping disaster,1392940800
6,5,True,"12 1, 2013",A94ZG5CWN70E7,1888861614,trinette claude,Omg.. The S was inexpensive and EXACTLY what w...,Great quality for this product. I have never b...,1385856000
7,3,True,"11 6, 2013",A3QBT8YC3CZ7C9,1888861614,Rela1982,It was a nice size for a cake topper and it wa...,Great Buy.,1383696000
8,5,True,"10 2, 2013",AGKW3A1RB1YGO,1888861614,"Marilyn Hatboro,Pa",Perfect!!! Can not wait to use on my Wedding c...,Very Pretty!!,1380672000
9,5,True,"09 6, 2013",A1B65IWLUVOUQB,1888861614,KASmith,This was exactly what we were looking for to m...,Perfect!,1378425600


In [4]:
Grocery_json.columns


Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime'],
      dtype='object')

In [5]:
#read labeled csv
labeled_df = pd.read_csv('Resources/labeled.csv')
labeled_df

,words,sentiment
0,"['comforting', 'know', 'processed', 'like', 's...",pos
1,"['never', 'disappointed', 'cup', 'cider', 'lik...",neg
2,"['never', 'ever', 'write', 'reviews', 'product...",neg
3,"['tasty', 'package', 'arrived', 'neatly', 'pac...",pos
4,"['gonna', 'buy', 'future', 'ate', 'completely'...",pos
...,...,...
809635,"['great', 'tasting', 'tuna', 'easy', 'open', '...",pos
809636,"['addicted', 'product', 'grocer', 'quit', 'car...",neg
809637,"['sure', 'happened', 'box', 'arrived', 'comple...",neg
809638,"['put', 'water', 'bottle', 'done', 'working', ...",pos


In [6]:
#count the number of words in a review
Grocery_json["reviewlength"] = labeled_df["words"].apply(lambda x: len(x))

Grocery_json.head(50)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,reviewlength
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,270
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,287
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,1156
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,126
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,92
5,1,True,"02 21, 2014",ATQL0XOLZNHZ4,1888861614,Brianna Small,"The S is beautiful, however upon checkout it i...",shipping disaster,1392940800,97
6,5,True,"12 1, 2013",A94ZG5CWN70E7,1888861614,trinette claude,Omg.. The S was inexpensive and EXACTLY what w...,Great quality for this product. I have never b...,1385856000,22
7,3,True,"11 6, 2013",A3QBT8YC3CZ7C9,1888861614,Rela1982,It was a nice size for a cake topper and it wa...,Great Buy.,1383696000,96
8,5,True,"10 2, 2013",AGKW3A1RB1YGO,1888861614,"Marilyn Hatboro,Pa",Perfect!!! Can not wait to use on my Wedding c...,Very Pretty!!,1380672000,57
9,5,True,"09 6, 2013",A1B65IWLUVOUQB,1888861614,KASmith,This was exactly what we were looking for to m...,Perfect!,1378425600,44


In [7]:
groceries_df = Grocery_json[['asin','overall','reviewerID', 'reviewTime','unixReviewTime', 'reviewText', "reviewlength"]]
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
7,1888861614,3,A3QBT8YC3CZ7C9,"11 6, 2013",1383696000,It was a nice size for a cake topper and it wa...,96
8,1888861614,5,AGKW3A1RB1YGO,"10 2, 2013",1380672000,Perfect!!! Can not wait to use on my Wedding c...,57
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44


In [8]:
#drop dupicates
groceries_df = groceries_df.drop_duplicates()
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
7,1888861614,3,A3QBT8YC3CZ7C9,"11 6, 2013",1383696000,It was a nice size for a cake topper and it wa...,96
8,1888861614,5,AGKW3A1RB1YGO,"10 2, 2013",1380672000,Perfect!!! Can not wait to use on my Wedding c...,57
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44


In [9]:
rating_5 = groceries_df.loc[groceries_df['overall'] == 5]
rating_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
8,1888861614,5,AGKW3A1RB1YGO,"10 2, 2013",1380672000,Perfect!!! Can not wait to use on my Wedding c...,57
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44


In [10]:
rating_1 = groceries_df.loc[groceries_df['overall'] == 1]
rating_1

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97


In [11]:
rating_2 = groceries_df.loc[groceries_df['overall'] == 2]
rating_2

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength


In [12]:
rating_3 = groceries_df.loc[groceries_df['overall'] == 3]
rating_3

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
7,1888861614,3,A3QBT8YC3CZ7C9,"11 6, 2013",1383696000,It was a nice size for a cake topper and it wa...,96


In [13]:
rating_4 = groceries_df.loc[groceries_df['overall'] == 4]
rating_4

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92


In [14]:
groceries_df_12_45 = groceries_df[groceries_df.overall != 3]
groceries_df_12_45

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
8,1888861614,5,AGKW3A1RB1YGO,"10 2, 2013",1380672000,Perfect!!! Can not wait to use on my Wedding c...,57
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44


In [15]:
#choosing every second row of the five star rating for a total of 600,000 to reduce its size from around 3.5 million ratings to a number closer to the 1 star ratings. 
rating_5_trimmed = rating_5[:1200000:2]
rating_5_trimmed

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44


In [16]:
groceries_df_1_5 = [rating_5_trimmed, rating_1]
groceries_df_1_5
groc_1_5 = pd.concat(groceries_df_1_5)
groc_1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97


In [17]:
g1_5 = pd.merge(rating_5_trimmed, rating_1, how='outer')
g1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
2,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44
3,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97


In [18]:
groceries_meta = pd.read_csv('Resources/fakemeat_meta.csv')
groceries_meta

,asin,brand,title,price
0,B000LKXRI6,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",NaN
1,B000LKZL4Y,Tofurky,"Turtle Island Tofurky Jurky, Ginger Teriyaki, ...",NaN
2,B000O6I9YY,Tofurky,"Tofurky, Deli Slices, Hickory Smoked, 5.5 oz",NaN
3,B000O6K870,Boca,Boca Original Chik'n Vegan Patties Made with N...,NaN
4,B000REIDYO,Quorn,"Quorn Chicken Style Tenders, 12 Oz (Pack of 12)",NaN
...,...,...,...,...
111,B01D9DHLNS,Simply Balanced,"Simply Balanced Fruit Strips Strawberry, Apric...",$25.99
112,B01DOAQ7KE,Tofurky,"Tofurky, Deli Slices, Bologna, 5.5 oz",NaN
113,B01DOB31XY,Tofurky,"Tofurky Slow Roasted Bbq Style Chick'n, 8 Ounc...",NaN
114,B01EVO3OAS,Tofurky,"Tofurky Veggie Burger Hearty Hemp, 10.5 Ounce ...",NaN


In [19]:
#groceries_meta = groceries_meta['price'].dropna()
#groceries_meta

In [20]:
#groceries_meta.count()

In [21]:
#groceries15merged = pd.merge(g1_5, groceries_meta, on='asin', how='outer')
groceries15merged = [g1_5, groceries_meta]
groceries15merged


[         asin  overall      reviewerID   reviewTime  unixReviewTime  \
 0  1888861614        5   ALP49FBWT4I7V   06 4, 2013      1370304000   
 1  1888861614        5   A94ZG5CWN70E7   12 1, 2013      1385856000   
 2  1888861614        5  A1B65IWLUVOUQB   09 6, 2013      1378425600   
 3  1888861614        1   ATQL0XOLZNHZ4  02 21, 2014      1392940800   
 
                                           reviewText  reviewlength  
 0  Very pleased with my purchase. Looks exactly l...           270  
 1  Omg.. The S was inexpensive and EXACTLY what w...            22  
 2  This was exactly what we were looking for to m...            44  
 3  The S is beautiful, however upon checkout it i...            97  ,
            asin            brand  \
 0    B000LKXRI6          Tofurky   
 1    B000LKZL4Y          Tofurky   
 2    B000O6I9YY          Tofurky   
 3    B000O6K870             Boca   
 4    B000REIDYO            Quorn   
 ..          ...              ...   
 111  B01D9DHLNS  Simply Bal

In [22]:
groc_1_5 = pd.concat(groceries15merged)
groc_1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength,brand,title,price
0,1888861614,5.0,ALP49FBWT4I7V,"06 4, 2013",1.370304e+09,Very pleased with my purchase. Looks exactly l...,270.0,NaN,NaN,NaN
1,1888861614,5.0,A94ZG5CWN70E7,"12 1, 2013",1.385856e+09,Omg.. The S was inexpensive and EXACTLY what w...,22.0,NaN,NaN,NaN
2,1888861614,5.0,A1B65IWLUVOUQB,"09 6, 2013",1.378426e+09,This was exactly what we were looking for to m...,44.0,NaN,NaN,NaN
3,1888861614,1.0,ATQL0XOLZNHZ4,"02 21, 2014",1.392941e+09,"The S is beautiful, however upon checkout it i...",97.0,NaN,NaN,NaN
0,B000LKXRI6,NaN,NaN,NaN,NaN,NaN,NaN,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",NaN
...,...,...,...,...,...,...,...,...,...,...
111,B01D9DHLNS,NaN,NaN,NaN,NaN,NaN,NaN,Simply Balanced,"Simply Balanced Fruit Strips Strawberry, Apric...",$25.99
112,B01DOAQ7KE,NaN,NaN,NaN,NaN,NaN,NaN,Tofurky,"Tofurky, Deli Slices, Bologna, 5.5 oz",NaN
113,B01DOB31XY,NaN,NaN,NaN,NaN,NaN,NaN,Tofurky,"Tofurky Slow Roasted Bbq Style Chick'n, 8 Ounc...",NaN
114,B01EVO3OAS,NaN,NaN,NaN,NaN,NaN,NaN,Tofurky,"Tofurky Veggie Burger Hearty Hemp, 10.5 Ounce ...",NaN


In [23]:
groceries_csv = groc_1_5.to_csv(path_or_buf='Resources/groceries_merged_trimmed.csv', index=False)


In [24]:
# sqlalchemy's create engine preparing the ability to connect to a sqlite database. then with .connect connecting to the database. 
engine = create_engine('sqlite:///groc15trimmed.sqlite', echo=True)
sqlite_connection = engine.connect()

In [25]:
sqlite_table = "groceriesALL"
ALLgroceries=groc_1_5.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-06-20 23:28:26,116 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-20 23:28:26,186 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-20 23:28:26,189 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-20 23:28:26,189 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-20 23:28:26,191 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-20 23:28:26,192 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-20 23:28:26,193 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("groceriesALL")
2021-06-20 23:28:26,194 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-20 23:28:26,197 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-20 23:28:26,198 INFO sqlalchemy.engine.Engine [raw sql] ('groceriesALL',)
2021-06-20 23:28:26,199 INFO sqlalchemy.engine.Engine PRAGMA main.foreig

In [26]:
#sqlite_connection.close()

In [27]:
# reflect an existing database into a new model
#transfer the contents of the database into a different structure of data. 
Base = automap_base()

In [28]:
# reflect the tables= creating classes that will keep the code separate, makes it possible to interact with only a subset of the data
#reflect the schema from the tables to our code
Base.prepare(engine, reflect=True)

2021-06-20 23:28:26,548 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-20 23:28:26,548 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-20 23:28:26,550 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("groceriesALL")
2021-06-20 23:28:26,551 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-20 23:28:26,555 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-20 23:28:26,555 INFO sqlalchemy.engine.Engine [raw sql] ('groceriesALL',)
2021-06-20 23:28:26,557 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("groceriesALL")
2021-06-20 23:28:26,558 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-20 23:28:26,559 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("groceriesALL")
2021-06-20 23:28:26,560 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-20 23:28:26,561 INFO sqlalchemy.engine.Engine SELECT

In [29]:
# Base.classes gives access to all the classes(=tables), .keys() references all the names of the classes
Base.classes.keys()

[]

In [30]:
# give a new variable to the groceries class instead of having to write base.classes.name of table use the variable every time you want to reference that class. 
GroceriesALL = Base.classes.groceriesALLe

AttributeError: groceriesALL

In [ ]:
# Session method called to start a session to query the database
session = Session(engine)

In [ ]:
results = session.query(groceriesALL.overall, groceriesALL.reviewText)

In [ ]:
#allGroc = Base.classes.